# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices. This Series should be sorted by a tie-break sort in the format of ("extracted date", "original row number").

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
import pandas as pd

doc = []
with open('assets/dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [19]:
def date_sorter():
    
    r1 = '(\d{1,2}[/-]\d{1,2}[/-]\d{2,4})'
    r2 = '((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[\S]*[+\s]\d{1,2}[,]{0,1}[+\s]\d{4})'
    r3 = '(\d{1,2}[+\s](?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[\S]*[+\s]\d{4})'
    r4 = '((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[\S]*[+\s]\d{4})'
    r5 = '(\d{1,2}[/-][1|2]\d{3})'
    r6 = '([1|2]\d{3})'
    pattern = f"({r1}|{r2}|{r3}|{r4}|{r5}|{r6})"

    parsed_date = df.str.extract(pattern, expand=True)
    parsed_date[6] = parsed_date[0].str.replace('Janaury', 'January').str.replace('Decemeber', 'December')

    parsed_date['date'] = pd.to_datetime(parsed_date[6])

    parsed_date['year'] = parsed_date['date'].dt.year
    parsed_date['month'] = parsed_date['date'].dt.month
    parsed_date['day'] = parsed_date['date'].dt.day

    parsed_date.loc[parsed_date['year'] == 2071, 'year'] = '1971'
    parsed_date.loc[parsed_date['year'] == 2072, 'year'] = '1972'

    parsed_date['format_date'] = parsed_date.apply(lambda row: f"{row['year']}-{row['month']}-{row['day']}", axis=1)

    ordened = pd.Series(pd.to_datetime(parsed_date['format_date'], format='%Y-%m-%d')).sort_values(ascending=True).index
    # ordened = ordened.sort_values(ascending=True).index
    # parsed_date['text'] = df
    # return parsed_date

    return pd.Series(ordened)

x = date_sorter()

C:\Users\bruno\AppData\Local\Temp\ipykernel_16848\3168716545.py:14: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  parsed_date['date'] = pd.to_datetime(parsed_date[6])


In [3]:
# parsed_date[parsed_date[0].str.contains(r'(/\d{2}$)')]

In [22]:
x

0        9
1       84
2        2
3       53
4       28
      ... 
495    231
496    141
497    186
498    161
499    413
Length: 500, dtype: int64

In [9]:
x.columns

Index([0, 1, 2, 3, 4, 5, 6, 'date', 'year', 'month', 'day', 'format_date',
       'text'],
      dtype='object')

In [11]:
y = x[['date', 'year', 'month', 'day', 'format_date', 'text']].copy()

In [24]:
r1 = '(\d{1,2}[/-]\d{1,2}[/-]\d{2,4})'
r2 = '((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[\S]*[+\s]\d{1,2}[,]{0,1}[+\s]\d{4})'
r3 = '(\d{1,2}[+\s](?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[\S]*[+\s]\d{4})'
r4 = '((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[\S]*[+\s]\d{4})'
r5 = '(\d{1,2}[/-][1|2]\d{3})'
r6 = '([1|2]\d{3})'
pattern = f"({r1}|{r2}|{r3}|{r4}|{r5}|{r6})"

parsed_date = df.str.extract(pattern, expand=True)
parsed_date[6] = parsed_date[0].str.replace('Janaury', 'January').str.replace('Decemeber', 'December')

parsed_date['date'] = pd.to_datetime(parsed_date[6])

parsed_date['year'] = parsed_date['date'].dt.year
parsed_date['month'] = parsed_date['date'].dt.month
parsed_date['day'] = parsed_date['date'].dt.day

parsed_date.loc[parsed_date['year'] == 2071, 'year'] = '1971'
parsed_date.loc[parsed_date['year'] == 2072, 'year'] = '1972'

parsed_date['format_date'] = parsed_date.apply(lambda row: f"{row['year']}-{row['month']}-{row['day']}", axis=1)

C:\Users\bruno\AppData\Local\Temp\ipykernel_16848\2755575145.py:12: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  parsed_date['date'] = pd.to_datetime(parsed_date[6])


In [35]:
parsed_date[parsed_date[0].str.contains(r'(/\d{2}$)')][60:70]

C:\Users\bruno\AppData\Local\Temp\ipykernel_16848\3309448080.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  parsed_date[parsed_date[0].str.contains(r'(/\d{2}$)')][60:70]


,0,1,2,3,4,5,6,date,year,month,day,format_date
79,10/04/98,10/04/98,NaN,NaN,NaN,NaN,10/04/98,1998-10-04,1998,10,4,1998-10-4
80,6/29/81,6/29/81,NaN,NaN,NaN,NaN,6/29/81,1981-06-29,1981,6,29,1981-6-29
81,8/04/78,8/04/78,NaN,NaN,NaN,NaN,8/04/78,1978-08-04,1978,8,4,1978-8-4
84,5/18/71,5/18/71,NaN,NaN,NaN,NaN,5/18/71,2071-05-18,1971,5,18,1971-5-18
86,6/05/93,6/05/93,NaN,NaN,NaN,NaN,6/05/93,1993-06-05,1993,6,5,1993-6-5
87,8/9/97,8/9/97,NaN,NaN,NaN,NaN,8/9/97,1997-08-09,1997,8,9,1997-8-9
88,12/8/82,12/8/82,NaN,NaN,NaN,NaN,12/8/82,1982-12-08,1982,12,8,1982-12-8
89,8/26/89,8/26/89,NaN,NaN,NaN,NaN,8/26/89,1989-08-26,1989,8,26,1989-8-26
90,10/13/95,10/13/95,NaN,NaN,NaN,NaN,10/13/95,1995-10-13,1995,10,13,1995-10-13
91,4/19/91,4/19/91,NaN,NaN,NaN,NaN,4/19/91,1991-04-19,1991,4,19,1991-4-19


In [34]:
parsed_date.loc[9]

0                          4/10/71
1                          4/10/71
2                              NaN
3                              NaN
4                              NaN
5                              NaN
6                          4/10/71
date           2071-04-10 00:00:00
year                          1971
month                            4
day                             10
format_date              1971-4-10
Name: 9, dtype: object